In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

In [2]:
display(Markdown("## INTERACTIVE EXPLANATION TOOL"))
intro_output = widgets.Output()
display(intro_output)
with intro_output:
    clear_output(wait=True)
    print("Initializing application...")
    
def print_intro(text):
    with intro_output:
        print(text)
        
def intro_done():
    intro_output.clear_output()

## INTERACTIVE EXPLANATION TOOL

Output()

In [3]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

# supress deprecation warnings from TF
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
###################

# Code deels gebaseerd op https://towardsdatascience.com/explainable-artificial-intelligence-part-3-hands-on-machine-learning-model-interpretation-e8ebe5afc608
# en op de github repo's van LIME en SHAP.

from IPython.display import display
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from lime.lime_tabular import LimeTabularExplainer
import xgboost
import shap
import time
import os.path
print_intro("Imported packages")

In [5]:
with intro_output:
    shap.initjs()

In [6]:
dataset = helpers.load_adult_income_dataset()
print_intro("Dataset loaded")

In [7]:
# Creating DiCE data object
continuous_features=['age', 'hours_per_week']
d = dice_ml.Data(dataframe=dataset, continuous_features=continuous_features, outcome_name='income')

In [8]:
############################
# Creating dataset for LIME
feature_names = dataset.columns.to_list()[:-1] # weggelaten: education_num, relationship, capital gain, capital loss, country
labels = dataset.iloc[:,-1].to_numpy()
data = dataset.iloc[:,:-1].to_numpy()
class_names = np.array(['<=50K', '>50K'])
categorical_features = [feature for feature in feature_names if feature not in continuous_features]

In [9]:
##################################
# Encode categorical features
categorical_features = [1,2,3,4,5,6]
categorical_names = {}
categorical_names_indexed_by_name = {}
for feature in categorical_features:
    le = LabelEncoder()
    le.fit(data[:, feature])
    data[:, feature] = le.transform(data[:, feature])
    categorical_names[feature] = le.classes_
    categorical_names_indexed_by_name[feature_names[feature]] = le.classes_
    
def category_number_to_name(instance):
    result = instance.astype('<U26')
    for categorical_index in categorical_names.keys():
        cat_label = int(instance[categorical_index])
        result[categorical_index] = categorical_names[categorical_index][cat_label]
    return result

def category_name_to_number(instance):
    result = np.copy(instance)
    for categorical_index in categorical_names.keys():
        number = np.where(categorical_names[categorical_index] == instance[categorical_index])
        result[categorical_index] = number[0][0]
    return result

In [10]:
#################################
# Splitting train and test
# using same random state for "data" (LIME) and "dataset (DiCE)"
random_state = 17
data = data.astype(float)
np.random.seed(1)
X_train_lime, X_test_lime, y_train_lime, y_test_lime = train_test_split(data, labels, random_state = random_state, 
                                                                        test_size=0.2)
print_intro("Dataset preprocessed")

In [11]:
# seeding random numbers for reproducability
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [12]:
# try to load model, if not exists: train model
model_name = "keras_ann_v1"
model_path = "../models/"+model_name+".h5"
sess = tf.InteractiveSession()

if (os.path.isfile(model_path)):
    ann_model = tf.keras.models.load_model(model_path)
    print_intro("Model loaded from disk")
    
else:
    print_intro("Training model")

    train, test = train_test_split(d.normalize_data(d.one_hot_encoded_data), random_state=random_state, test_size=0.2)
    X_train = train.loc[:, train.columns != 'income']
    y_train = train.loc[:, train.columns == 'income']

    X_test = test.loc[:, test.columns != 'income']
    y_test = test.loc[:, test.columns == 'income']

    ann_model = keras.Sequential()
    ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l1(0.001), activation=tf.nn.relu))
    ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

    ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
    ann_model.fit(X_train, y_train, validation_split=0.20, epochs=100, verbose=1, class_weight={0:1,1:2})
    print("accuracy: "+str(ann_model.history.history['acc'][-1]))
    # the training will take some time for 100 epochs.
    # you can wait or set verbose=1 to see the progress of training.
    
    # save model
    ann_model.save("../models/"+model_name+".h5")
    print_intro("Model saved to disk")

In [13]:
def inverse_categories(instances):
    result = instances.astype('<U26')
    for categorical_index in categorical_names.keys():
        cat_labels = instances[:,categorical_index]
        result[:,categorical_index] = [categorical_names[categorical_index][int(cat_label)] for cat_label in cat_labels]
    return result

def lime_instances_to_dice_ml(instances):
    base_frame = d.prepare_df_for_encoding()
    
    instances_categorical = inverse_categories(instances)

    instance_dataframe = pd.DataFrame(instances_categorical,columns=feature_names)
    instance_dataframe['age'] = pd.to_numeric(instance_dataframe['age'])
    instance_dataframe['hours_per_week'] = pd.to_numeric(instance_dataframe['hours_per_week'])

    """Prepares user defined test input for DiCE."""
    temp = base_frame.append(instance_dataframe, ignore_index=True, sort=False)
    temp = d.one_hot_encode_data(temp)
    temp = d.normalize_data(temp)
    final = temp.tail(instance_dataframe.shape[0]).reset_index(drop=True)
    return final

def lime_instance_to_dict(instance):
    temp = dict(zip(feature_names,category_number_to_name(instance)))
    temp['age'] = int(float(temp['age']))
    temp['hours_per_week'] = int(float(temp['hours_per_week']))
    return temp

def dict_to_lime_instance(instance_dict):
    instance_array = np.array(list(instance_dict.values()))
    return category_name_to_number(instance_array).astype('float')

def pd_to_dataframe(instance):
    return pd.DataFrame(instance.reshape(-1, len(instance)),columns=feature_names)

def predict_fn_lime_superquick(instances):
    converted_to_dice = lime_instances_to_dice_ml(instances)
    predictions = ann_model.predict(converted_to_dice)
    return np.append(1-predictions, predictions, axis=1)

def predict_fn_shap_superquick(instances):
    converted_to_dice = lime_instances_to_dice_ml(instances)
    predictions = ann_model.predict(converted_to_dice)
    return np.array([pred[0] for pred in predictions])

In [14]:
# provide the trained ML model to DiCE's model object
backend = 'TF'+tf.__version__[0] # TF1
m = dice_ml.Model(model=ann_model, backend=backend) 

In [15]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [16]:
###############################
explainer_lime = LimeTabularExplainer(X_train_lime ,feature_names = feature_names,class_names=class_names,
                                                   categorical_features=categorical_features, 
                                                   categorical_names=categorical_names, kernel_width=3)

In [17]:
%%capture 
# %%capture dient om een vervelende warning te onderdrukken (doet alle output van deze cell weg)
############################## SHAP

# Median is simpelweg de mediaan van elke feature in de training set. In deze setting wordt telkens de mediaan van een feature als baseline gebruikt (ze krijgt deze waarde bij het "wegdoen")
# INFO: De base value op de figuur is de uitkomst die de blackbox geeft voor de mediaan als input.
med = np.median(X_train_lime, axis=0).reshape((1,X_train_lime.shape[1]))

explainer_shap = shap.KernelExplainer(predict_fn_shap_superquick, med)

def plot_explanation_for_shap(instance):
    shap_values_for_instance = explainer_shap.shap_values(instance, nsamples=1000)
    return shap.force_plot(explainer_shap.expected_value, shap_values_for_instance, category_number_to_name(instance), feature_names = feature_names)

In [18]:
print_intro("Ready!")

In [19]:
# helper functions

def get_prediction_prob(instance):
    return predict_fn_shap_superquick(np.array([instance]))[0]

def get_sum_of_negative_weights(lime_explanation):
    total = 0
    for element in lime_explanation.as_list():
        if element[1] < 0:
            total += abs(element[1])
    return total

def get_most_important_negative_features(lime_explanation, probability=0.5, min_features = 2, max_features=4):
    features_to_show = []
    weight_so_far = 0
    total_weight = get_sum_of_negative_weights(lime_explanation)
    for element in lime_explanation.as_list():
        if element[1] > 0:
            continue
        if len(features_to_show) >= max_features:
            break
        if len(features_to_show) >= min_features and weight_so_far / total_weight > probability:
            break
        features_to_show.append(element[0])
        weight_so_far += abs(element[1])
    return features_to_show

def get_all_negative_features(lime_explanation):
    features_to_show = []
    for element in lime_explanation.as_list():
        if element[1] < 0:
            features_to_show.append(element[0])
    return features_to_show

def get_remaining_negative_features(lime_explanation):
    most_important = get_most_important_negative_features(lime_explanation)
    all_negative = get_all_negative_features(lime_explanation)
    return all_negative[len(most_important):]

def print_explanation_step_1(lime_explanation):
    print("In your case, the most important reasons for making LESS than 50K/year are (in descending order of importance):")
    features = get_most_important_negative_features(lime_explanation)
    for element in features:
        display(Markdown("* "+str(element)))
        
def print_explanation_step_2(lime_explanation):
    print("The other reasons for making LESS than 50K/year are:")
    for element in get_remaining_negative_features(lime_explanation):
        display(Markdown("* "+str(element)))

In [20]:
# Methods for initializing toggles and sliders on counterfactual page
def initialize_adjusters(instance):
    feature_adjusters = {}
    style = {'description_width': '100px'}
    for feature in feature_names:
        if feature in continuous_features:
            feature_range = d.get_features_range()[feature]
            adjuster = widgets.IntSlider(value=int(float(instance[feature])), min=feature_range[0], max=feature_range[1], step=1, description=feature, disabled=False, 
                                         continuous_update=False, orientation='horizontal', readout=True, readout_format='d', style=style, layout=Layout(width='425px'))
        else:
            subcategories = categorical_names_indexed_by_name[feature]
            adjuster = widgets.Dropdown(options=subcategories, value=str(instance[feature][0]), description=feature, disabled=False, style=style, layout=Layout(width='400px'))
            
        feature_adjusters[feature] = adjuster
    return feature_adjusters

def link_adjusters_to_callback(adjusters, callback):
    for adjuster in adjusters.values():
        adjuster.observe(callback, names='value')

def get_adjusters_values(adjusters):
    return {item[0]:item[1].value for item in adjusters.items()}

def get_dataframe_from_adjusters(adjusters):
    values = list(get_adjusters_values(adjusters).values())
    return pd.DataFrame([values],columns=feature_names)

def set_adjuster_values(adjusters, values):
    for feature in feature_names:
        if feature in continuous_features:
            adjusters[feature].value = int(float(values[feature]))
        else:
            adjusters[feature].value = str(values[feature][0])

In [21]:
def initialize_toggles(instance):
    feature_toggles = {}
    style = {'description_width': '100px'}
    for feature in feature_names: #info, warning, success, danger
        toggle = widgets.ToggleButton(value=False, description='flexible', disabled=False, button_style='info',
                                    tooltip='Click to lock this feature', icon='unlock') # (FontAwesome names without the `fa-` prefix)
        toggle.observe(on_toggle_click, 'value')
        feature_toggles[feature] = toggle
    return feature_toggles

def get_toggle_button_name(toggle):
    for feature in toggles:
        if toggles[feature] == toggle:
            return feature

def on_toggle_click(d):
    t = d['owner']
    if t.value == True:
        t.description = "fixed"
        t.icon = "lock"
        t.tooltip = "Click to make flexible"
        t.button_style = "danger"
    else:
        t.description = "flexible"
        t.icon = "unlock"
        t.tooltip = "Click to make fixed"
        t.button_style = "info"
    feature_name = get_toggle_button_name(t)
    if feature_name in continuous_features:
        hide_slider(feature_name, t.value)
        
def initialize_sliders(instance):
    feature_sliders = {}
    for feature in continuous_features:
        slider_range = d.get_features_range()[feature]
        slider = widgets.IntRangeSlider(value=slider_range, min=slider_range[0], max=slider_range[1], step=1, description=feature, disabled=False, continuous_update=False, 
                                orientation='horizontal', readout=True, readout_format='d', style = {'description_width': '95px'},layout=Layout(width='330px'))
        feature_sliders[feature] = slider
    return feature_sliders

def hide_slider(feature_name, hide):
    slider = sliders[feature_name]
    if hide:
        slider.layout.visibility = 'hidden'
    else:
        slider.layout.visibility = 'visible'
    
def get_toggle_label_box(toggles):
    vertical_list = []
    for feature in feature_names:
        label_layout = Layout(width='100px')
        label = widgets.Label(value=feature, layout=label_layout)
        hbox = widgets.HBox([label,toggles[feature]])
        vertical_list.append(hbox)
    return widgets.VBox(vertical_list)

def get_features_to_vary():
    result = []
    for feature in feature_names:
        if toggles[feature].value == False:
            result.append(feature)
    return result
            
def get_feature_ranges():
    new_ranges = {}
    for feature in continuous_features:
        new_ranges[feature] = sliders[feature].value
    return new_ranges

In [22]:
intro_done()
test = pd_to_dataframe(category_number_to_name(X_test_lime[2,:]))

In [25]:
############ Main script ###################
class App_state:
    def __init__(self):
        self.lime_expl = None
        self.shap_expl = None
        self.instance_dataframe = None
        self.instance_lime = None
        
app_state = App_state()

# Create widget objects (adjusters and output panes)
adjusters = initialize_adjusters(test)
output_information = widgets.Output(layout={'border': '1px solid black'})
output_settings = widgets.Output(layout={'border': '1px solid black'})
output_prediction = widgets.Output()
output_probability = widgets.Output()
output_explanation = widgets.Output()
output_counterfactuals = widgets.Output()
output_constraints = widgets.Output(layout={'border': '1px solid black'})
output_dice = widgets.Output(layout={'border': '1px solid black'})
output_bin = widgets.Output()

# Reset (because of changes in personal features)
def adjuster_changed(change):
    output_explanation.layout = {}
    with output_counterfactuals:
        clear_output(wait=False)
    with output_explanation:
        clear_output(wait=False)
    with output_probability:
        clear_output(wait=False)
    with output_prediction:
        clear_output(wait=False)
    button_predict.layout.display = "block"
    
link_adjusters_to_callback(adjusters, adjuster_changed)

# Reset button
def reset_adjusters(b):
    set_adjuster_values(adjusters,test)
    adjuster_changed(None) #dummy value passed
    with output_prediction:
        print("Reset to default")

button_reset = widgets.Button(description="Reset", button_style = 'warning')
button_reset.on_click(reset_adjusters)

# Predict button
def on_button_predict_clicked(b):
    button_predict.layout.display = "none"
    app_state.instance_dataframe = get_dataframe_from_adjusters(adjusters)
    app_state.instance_lime = category_name_to_number(app_state.instance_dataframe.to_numpy()[0]).astype('float')
    button_counterfactuals.layout.visibility = 'visible'
         
    with output_prediction:
        clear_output(wait=True)
        pred_prob = get_prediction_prob(app_state.instance_lime)
        if pred_prob > 0.5:
            display(Markdown("#### <br>__Prediction: you earn MORE than 50k/year__<br>"))
            print("This tool currently only provides explanations when the AI predicts LESS than 50k/year. (Besides, with this result, no explanation needed right ?)")
            print("To generate explanations, try out some different combinations above, or use the default exmaple by clicking the RESET button.")
        else:
            display(Markdown("#### <br>__Prediction: you earn LESS than 50k/year__<br>"))
            with output_probability:
                clear_output(wait=True)
                button_confidence.layout.display = "block"
                display(button_confidence)
                print()
            with output_explanation:
                clear_output(wait=True)
                button_explain.layout.display = "block"
                display(button_explain)
                print()
            with output_counterfactuals:
                clear_output(wait=True)
                button_counterfactuals.layout.display = "block"
                display(button_counterfactuals)

button_predict = widgets.Button(description="Predict outcome", button_style = 'primary')
button_predict.on_click(on_button_predict_clicked)

# Confidence button
def on_button_confidence_clicked(b):
    button_confidence.layout.display = "none"
    with output_probability:
        pred_prob = get_prediction_prob(app_state.instance_lime)
        if pred_prob > 0.5:
            print("The AI is "+str(round(pred_prob*100,0))+"% sure.")
        else:
            print("The AI is "+str(round((1-pred_prob)*100,0))+"% sure.")
        print() #newline

button_confidence = widgets.Button(description="How confident is the AI ?", button_style = 'primary', layout=Layout(width='auto'))   
button_confidence.on_click(on_button_confidence_clicked)

# Explain button
def on_button_explain_clicked(b):
    button_explain.layout.display = "none"
    output_explanation.layout = {'border': '1px solid black'}
    disable_all_widgets(True)
    with output_explanation:
            clear_output(wait=True)
    with output_bin:
        app_state.lime_expl = explainer_lime.explain_instance(app_state.instance_lime, predict_fn_lime_superquick, num_features=10)
        app_state.shap_expl = explainer_shap.shap_values(app_state.instance_lime, nsamples=1000)
    with output_explanation:
        display(Markdown("#### __Explanation__"))
        print_explanation_step_1(app_state.lime_expl)
        remaining_negatives = get_remaining_negative_features(app_state.lime_expl)
        if len(remaining_negatives) > 0:
            button_show_more.description ="Show "+str(len(remaining_negatives))+" more"
            button_show_more.layout.display = "block"
            display(button_show_more)
        else:
            button_lime.layout.display = "block"
            display(button_lime)
    disable_all_widgets(False)
        
button_explain = widgets.Button(description="Explain why the AI predicts LESS than 50K/year.", button_style = 'primary', layout=Layout( width='auto'))
button_explain.on_click(on_button_explain_clicked)

# Show more button
def on_button_show_more_clicked(b):
    button_show_more.layout.display = "none"
    with output_explanation:
        print_explanation_step_2(app_state.lime_expl)
        button_lime.layout.display = "block"
        display(button_lime)
        
button_show_more = widgets.Button(description="Show more..", button_style = 'primary', layout=Layout(width='auto'))   
button_show_more.on_click(on_button_show_more_clicked)

# Show all (LIME)
def on_button_lime_clicked(b):
    button_lime.layout.display = "none"
    with output_explanation:
        print("The graph below illustrates the expected behaviour of the model for people similar to you.")
        print("Blue bars indicate properties expected to lower the probability of earning more than 50K/year. Orange bars indicate the opposite.")
        print("The size of the bars determine the approximate impact of a property.")
        print()
        app_state.lime_expl.show_in_notebook(show_all=False, predict_proba=False)
        print("Note that this information is based on a local linear approximation of the AI.")
        print("The weights corresponding to the bars illustrate the behaviour of the linear approximation, but are in no way exact.")
        print()
        button_shap.layout.display = "block"
        display(button_shap)
        
button_lime = widgets.Button(description="Help me understand the rationale of the model.", button_style = 'primary', layout=Layout(width='auto'))   
button_lime.on_click(on_button_lime_clicked)

# Show impact (SHAP)
def on_button_shap_clicked(b):
    button_shap.layout.display = "none"
    with output_explanation:
        print("The plot below illustrates the impact of every property on this specific decision.")
        print()
        display(shap.force_plot(explainer_shap.expected_value, app_state.shap_expl, category_number_to_name(app_state.instance_lime), feature_names = feature_names))
        print("How to interpret these results:")
        display(Markdown("* The model output value corresponds to the probability of you earning MORE than 50k, predicted by the AI."))
        display(Markdown("* The base value indicates the AI's prediction for an average person (based on the samples it was trained on)"))
        display(Markdown("* Red bars illustrate which properties lead to a higher probability, compared to the base value. Blue bars indicate the opposite."))
        display(Markdown("* The total resultant force of the blue and red bars combined, correspond exactly to the difference between your prediction and the average prediction."))
        print("Note that, again, these results are based upon approximation")
        print("Moreover, the size of the bars DO NOT tell you how much the probability will change when a property would have been different.")
        
button_shap = widgets.Button(description="Help me better understand the impact of every property, regarding this specific decision.", button_style = 'primary', layout=Layout(width='auto'))   
button_shap.on_click(on_button_shap_clicked)

# Counterfactual button
def on_button_counterfactuals_clicked(b):
    button_counterfactuals.layout.display = "none"
    with output_counterfactuals:
        clear_output(wait=True)
        display(widgets.HBox([output_constraints,output_dice]))
    load_constraints()
    load_counterfactuals_dice()
    
button_counterfactuals = widgets.Button(description="Help me find out how the prediction would have been different.", button_style = 'primary', layout=Layout(width='auto'))
button_counterfactuals.on_click(on_button_counterfactuals_clicked)

# Search button
def on_button_search_clicked(b):
    load_counterfactuals_dice()
        
button_search = widgets.Button(description="Search again within constraints", button_style = 'primary', layout=Layout(width='320px'))
button_search.on_click(on_button_search_clicked)

def load_constraints():
    with output_constraints:
        clear_output(wait=True)
        display(Markdown("#### __Constraints__"))
        print("With the settings below, you can define which\nproperties should be kept fixed. For numerical\nproperties, you can define the allowed range of\nchange.")
        print("--------------------------------------")
        display(final_box)

def load_counterfactuals_dice():
    disable_all_widgets(True)
    with output_dice:
        clear_output()
        display(Markdown("#### __Counterfactuals__"))
        print("This part of the tool displays four alternative situations (called counterfactuals), under\nwhich the model would have predicted earning MORE than 50k/year.")
        print("Feel free to set some constraints on certain properties (perhaps you're not willing to\nchange your gender?). After setting new constraints, press the SEARCH AGAIN button.")
        print("------------------------------------------------------------------------------")
        print("\nLoading counterfactuals...")
    d.permitted_range = get_feature_ranges()
    exp = dice_ml.Dice(d, m)
    with output_bin:
        dice_exp = exp.generate_counterfactuals(lime_instance_to_dict(app_state.instance_lime), total_CFs=4, desired_class="opposite", features_to_vary=get_features_to_vary())
    with output_dice:
        clear_output(wait=True)
        display(Markdown("#### __Counterfactual outcomes__"))
        print("This part of the tool displays four alternative situations (called counterfactuals), under\nwhich the model would have predicted earning MORE than 50k/year.")
        print("Feel free to set some constraints on certain properties (perhaps you're not willing to\nchange your gender?). After setting new constraints, press the SEARCH AGAIN button.")
        print("------------------------------------------------------------------------------")
        original = dice_exp.org_instance
        counterfactuals = dice_exp.final_cfs_df_sparse
        for index,cf in counterfactuals.iterrows():
            output = counterfactual_table_outputs[index]
            md_table = "| {} |  |  |  |\n| --- | --- | --- | --- |".format(str(index+1))
            for feature in feature_names:
                if (original[feature][0] != cf[feature]):
                    md_table += "\n| __{}__ | {} | --> | {} |".format(feature,original[feature][0],cf[feature])
            with output:
                clear_output(wait=True)
                display(Markdown(md_table))
        display(table_grid)
    disable_all_widgets(False)
        
def disable_all_widgets(disable):
    button_reset.disabled = disable
    button_predict.disabled = disable
    button_confidence.disabled = disable
    button_explain.disabled = disable
    button_show_more.disabled = disable
    button_lime.disabled = disable
    button_shap.disabled = disable
    button_counterfactuals.disabled = disable
    button_search.disabled = disable
    for adjuster in adjusters.values():
        adjuster.disabled = disable
    for toggle in toggles.values():
        toggle.disabled = disable
    for slider in sliders.values():
        slider.disabled = disable

toggles = initialize_toggles(test)
sliders = initialize_sliders(test)
toggle_box = get_toggle_label_box(toggles)
final_list = [toggle_box]
final_list.extend(list(sliders.values()))
final_list.append(button_search)
final_box = widgets.VBox(final_list)

counterfactual_table_outputs = [widgets.Output() for i in range(4)]
table_grid = widgets.GridBox(counterfactual_table_outputs, layout=widgets.Layout(grid_template_columns="repeat(2, 350px)"))

adjusters_list = list(adjusters.values())
adjusters_list.append(widgets.HBox([button_reset,button_predict]))

with output_information:
    display(Markdown("#### __Instructions__"))
    print("---------------------------------------------------------")
    display(Markdown("A black-box AI, predicts whether or not you make $50.000 per <br />year, based on some personal information. Since black-boxes are <br />uninterpretable, this tool helps you find out WHY."))
    display(Markdown("Get started by initializing the settings on the right and pressing<br />the PREDICT OUTCOME button."))
    display(Markdown("Predictions are based on the [adult](https://archive.ics.uci.edu/ml/datasets/adult) dataset."))
    display(Markdown("The explanations are built using the [LIME](https://github.com/marcotcr/lime), [SHAP](https://github.com/slundberg/shap) and [DiCE](https://github.com/interpretml/DiCE) <br />packages. All credit goes to\nthe original authors."))
    display(Markdown("The source code of this tool is freely available on [GitHub](https://github.com/siebediels/thesis-demo)."))
with output_settings:
    display(Markdown("#### __Personal information__"))
    print("---------------------------------------------------------")
    display(widgets.VBox(adjusters_list))

display(widgets.HBox([output_information, output_settings]))
display(output_prediction)
display(output_probability)
display(output_explanation)
display(output_counterfactuals)

Output()

Output()

Output()

Output()